In [1]:
# Import dependencies
import pandas as pd
import plotly.graph_objects as go

## Data Processing

In [2]:
# Read in Holmes' game logs
holmes_df = pd.read_csv('Resources/Data/holmes_logs.csv')
holmes_df.head()

,Rk,Gtm,Date,IP,H,R,ER,BB,SO,HR,HBP,ERA,FIP,AVG,WHIP
0,1,1,Apr 8,0.2,3,1,1,0,0,0,0,13.50,3.11,0.600,4.50
1,2,2,Apr 9,1.0,0,0,0,1,1,0,0,5.40,3.71,0.375,2.40
2,3,5,Apr 12,1.2,0,0,0,0,2,0,0,2.70,2.21,0.231,1.20
3,4,7,Apr 14,0.2,1,0,0,0,1,0,0,2.25,1.86,0.250,1.25
4,5,9,Apr 16,1.1,0,0,0,0,1,0,0,1.69,1.80,0.200,0.94


In [3]:
# Create SO/IP colum
holmes_df['totalSO'] = holmes_df['SO'].cumsum()
holmes_df['floatIP'] = holmes_df['IP'].apply(lambda x: int(x) + (x % 1)*10/3)
holmes_df['totalIP'] = holmes_df['floatIP'].cumsum()
holmes_df['SO/IP'] = holmes_df['totalSO'] / holmes_df['totalIP']
holmes_df = holmes_df.round({'floatIP': 2, 'totalIP': 2, 'SO/IP': 2})
holmes_df.head()

,Rk,Gtm,Date,IP,H,R,ER,BB,SO,HR,HBP,ERA,FIP,AVG,WHIP,totalSO,floatIP,totalIP,SO/IP
0,1,1,Apr 8,0.2,3,1,1,0,0,0,0,13.50,3.11,0.600,4.50,0,0.67,0.67,0.00
1,2,2,Apr 9,1.0,0,0,0,1,1,0,0,5.40,3.71,0.375,2.40,1,1.00,1.67,0.60
2,3,5,Apr 12,1.2,0,0,0,0,2,0,0,2.70,2.21,0.231,1.20,3,1.67,3.33,0.90
3,4,7,Apr 14,0.2,1,0,0,0,1,0,0,2.25,1.86,0.250,1.25,4,0.67,4.00,1.00
4,5,9,Apr 16,1.1,0,0,0,0,1,0,0,1.69,1.80,0.200,0.94,5,1.33,5.33,0.94


In [4]:
# Filter for necessary columns
holmes_df = holmes_df[['Rk', 'Gtm', 'Date', 'ERA', 'FIP', 'AVG', 'WHIP', 'SO/IP']]
holmes_df.head()

,Rk,Gtm,Date,ERA,FIP,AVG,WHIP,SO/IP
0,1,1,Apr 8,13.50,3.11,0.600,4.50,0.00
1,2,2,Apr 9,5.40,3.71,0.375,2.40,0.60
2,3,5,Apr 12,2.70,2.21,0.231,1.20,0.90
3,4,7,Apr 14,2.25,1.86,0.250,1.25,1.00
4,5,9,Apr 16,1.69,1.80,0.200,0.94,0.94


In [5]:
# Convert Date string to datetime
holmes_df['Date'] = holmes_df['Date'] + ' 2022'
holmes_df['Date'] = pd.to_datetime(holmes_df['Date'], format="%b %d %Y")
holmes_df.dtypes

Rk                int64
Gtm               int64
Date     datetime64[ns]
ERA             float64
FIP             float64
AVG             float64
WHIP            float64
SO/IP           float64
dtype: object

In [6]:
# Filter for games after Holmes became closer
holmes_df = holmes_df.loc[holmes_df['Date'] >= '2022-05-15']
holmes_df.head()

,Rk,Gtm,Date,ERA,FIP,AVG,WHIP,SO/IP
17,18,34,2022-05-15,0.49,1.64,0.185,0.76,0.98
18,19,37,2022-05-18,0.44,1.69,0.183,0.74,0.93
19,20,39,2022-05-21,0.42,1.59,0.173,0.69,0.97
20,21,43,2022-05-24,0.40,1.48,0.167,0.66,1.01
21,22,44,2022-05-25,0.38,1.46,0.181,0.72,1.01


## Viz

In [95]:
# Create variables for use in chart
x = holmes_df['Date']
y1 = holmes_df['FIP']
y2 = holmes_df['ERA']
y3 = holmes_df['WHIP']

navy = '#0C2340'
gray = '#C4CED3'


# Create traces
fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y1, name='FIP', mode='lines+markers', line_color='#db5f4f'))
fig.add_trace(go.Scatter(x=x, y=y2, name='ERA', mode='lines+markers', line_color='#d4db6b'))
fig.add_trace(go.Scatter(x=x, y=y3, name='WHIP', mode='lines+markers', line_color='#4fbfdb'))


# Add traces for annotations and text for end of lines
for i, d in enumerate(fig.data):
    fig.add_trace(go.Scatter(x=[d.x[-1]],
                             y=[d.y[-1]],
                             mode='markers+text',
                             text=['<b>' + str(y) + '</b>' for y in [d.y[-1]]],
                             textfont = dict(
                                 size=18,
                                 color='white'),
                             textposition='middle right',
                             marker = dict(
                                 color = d.line.color,
                                 line=dict(
                                     width=d.marker.line.width,
                                     color=d.marker.line.color)),
                             showlegend=False))


# Format traces
fig.update_traces(
    marker=dict(
        size=22,
        line=dict(
            width=2,
            color='white')),
    line=dict(width=12)
)


# Format axes
fig.update_xaxes(
    tickmode='array',
    ticktext=['<b>' + date + '</b>' for date in ['May 15','May 29','Jun 12','Jun 26','Jul 10','Jul 24','Aug 7']],
    tickvals=['2022-05-15','2022-05-29','2022-06-12','2022-06-26','2022-07-10','2022-07-24','2022-08-07'],
    ticks= "outside",
    tickcolor= gray, 
    ticklen=8, 
    minor=dict(
        ticklen=4,  
        dtick=2*7*24*60*60*1000,
        tick0="2022-05-22"), 
    tickfont=dict(
        size=18,
        color=gray),
    showgrid=False
)

fig.update_yaxes(
    tickfont=dict(
        size=18,
        color=gray),
    tickmode='array',
    ticktext=['<b>' + x + '</b>' for x in ['0.00','0.50','1.00','1.50','2.00','2.50','3.00']],
    tickvals=[0, 0.5, 1, 1.5, 2, 2.5, 3],
    zerolinecolor='rgb(20,45,75)',
    zerolinewidth=2,
    gridcolor='rgb(20,45,75)',
    gridwidth=2,
)


# Format layout
fig.update_layout(
    title=dict(
        text="<b>Clay Holmes</b><br>Recent Decline in Performance",
        y=0.9,
        x=0.5,
        xanchor='center',
        yanchor='top',
        font=dict(
            size=28,
            color='white')),
    xaxis_range=['2022-05-10','2022-08-13'],
    yaxis_range=[0,3.1],
    legend=dict(
        title_text='',
        orientation='h',
        yanchor="top",
        y=0.947,
        xanchor="left",
        x=0.345,
        font=dict(
            size=18,
            color='white'),
        bordercolor=gray,
        borderwidth=2,
        bgcolor=navy),
    paper_bgcolor=navy,
    plot_bgcolor=navy,
    margin=dict(b=70, pad=3)
)